In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF
import pyspark.sql.functions as psf
from pyspark.sql.types import DoubleType

In [2]:
spark = SparkSession.builder.appName('TF-IDF').getOrCreate()
sc = spark.sparkContext

22/03/18 10:59:52 WARN Utils: Your hostname, adneovrebo.local resolves to a loopback address: 127.0.0.1; using 152.94.128.100 instead (on interface en0)
22/03/18 10:59:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/18 10:59:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
def split_file(x):
    value=x.split('\t')
    return (value[0], value[1].split(' '))

In [77]:
# Read categories into dataframe
categories = spark.read.csv('categories.csv', header=True, inferSchema=True)

# Make a column for each unique category, split each category on spaces
unique_cats = categories.select('categories').distinct()

# Split unique categories on spaces
cats_split = unique_cats.select(psf.split(psf.col('categories'), ' ').alias('categories'))

# Make a list of all the categories
cats_list = cats_split.select(psf.explode(psf.col('categories')).alias('category'))

# # Add to categories dataframe a column for each unique category
for row in cats_list.collect():
    category = row.category
    categories = categories.withColumn(category, psf.when(categories['categories'].contains(category), 1).otherwise(0))

In [78]:
data = (sc.textFile('cleaned-test.txt')
    .map(lambda x: split_file(x))
    .toDF(['id', 'words'])
)

data.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  3|["this, article, ...|
|  2|["lorem, ipsum, d...|
|  1|["lorem, ipsum, d...|
+---+--------------------+



In [80]:
# Join categories with data
joined_data = data.join(categories, ["id"])
joined_data.show()

+---+--------------------+--------------+--------------+-------+-----+------+
| id|               words|    categories|physics.gen-ph|math.CO|cs.CG|hep-ph|
+---+--------------------+--------------+--------------+-------+-----+------+
|  3|["this, article, ...|physics.gen-ph|             1|      0|    0|     0|
|  2|["lorem, ipsum, d...| math.CO cs.CG|             0|      1|    1|     0|
|  1|["lorem, ipsum, d...|        hep-ph|             0|      0|    0|     1|
+---+--------------------+--------------+--------------+-------+-----+------+



In [23]:
hashingTF = HashingTF(inputCol="words", outputCol='features', numFeatures=100000)
tf = hashingTF.transform(data)

In [21]:
idf = IDF(inputCol='features', outputCol='idf')
model = idf.fit(tf)
tf_idf = model.transform(tf)

In [22]:
cosine_similarity_udf = psf.udf(lambda x,y: round(float(x.dot(y)/(x.norm(2)*y.norm(2))), 2), DoubleType())

res = (tf_idf.alias("i").join(tf_idf.alias("j"), psf.col("i.id") < psf.col("j.id"))
    .select(
        psf.col("i.id").alias("i"), 
        psf.col("j.id").alias("j"), 
        cosine_similarity_udf("i.idf", "j.idf").alias("cosine_similarity"))
    .sort("i", "j"))
res.take(20)

ERROR:root:KeyboardInterrupt while sending command.===>           (10 + 1) / 36]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 